In [21]:
import numpy as np




filename = 'glove.6B.100d.txt'

def loadEmbeddings(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded!')
    file.close()
    return vocab,embd
vocab,embd = loadEmbeddings(filename)


word_vec_dim = len(embd[0])

vocab.append('<UNK>')
embd.append(np.asarray(embd[vocab.index('unk')],np.float32)+0.01)

vocab.append('<EOS>')
embd.append(np.asarray(embd[vocab.index('eos')],np.float32)+0.01)

vocab.append('<PAD>')
embd.append(np.zeros((word_vec_dim),np.float32))

embedding = np.asarray(embd)
embedding = embedding.astype(np.float32)



Loaded!


In [22]:
def word2vec(word):  # converts a given word into its vector representation
    if word in vocab:
        return embedding[vocab.index(word)]
    else:
        return embedding[vocab.index('<UNK>')]

def most_similar_eucli(x):
    xminusy = np.subtract(embedding,x)
    sq_xminusy = np.square(xminusy)
    sum_sq_xminusy = np.sum(sq_xminusy,1)
    eucli_dists = np.sqrt(sum_sq_xminusy)
    return np.argsort(eucli_dists)

word = 'frog'

most_similars = most_similar_eucli(word2vec(word))

print("TOP TEN MOST SIMILAR WORDS TO '"+str(word)+"':\n")
for i in range(0,10):
    print(str(i+1)+". "+str(vocab[most_similars[i]]))

def vec2word(vec):   # converts a given vector representation into the represented word 
    most_similars = most_similar_eucli(np.asarray(vec,np.float32))
    return vocab[most_similars[0]]

TOP TEN MOST SIMILAR WORDS TO 'frog':

1. frog
2. toad
3. snake
4. ape
5. monkey
6. frogs
7. litoria
8. spider
9. hypsiboas
10. squirrel


In [23]:
import string
# Data related to basic induction training and testing from QA bAbi tasks dataset will be used.
# (https://research.fb.com/downloads/babi/)

filename = 'qa16_basic-induction_train.txt' 

fact_story = [] 
question = []   
answer = []   

    
max_fact_len = 0
max_question_len = 0


def extract_info(filename,max_fact_len,max_question_len):  
        
    fact_story = [] 
    fact_stories = []
    questions = []
    answers = []

    file = open(filename,'r')
    for line in file.readlines(): 
        
        flag_end_story = 0 
        line = line.lower() 
        if '?' in line:
            #q for question, a for answer.
            flag_end_story=1
            qa = line.strip().split('\t')
            q = qa[0]
            a = qa[1]
            punct_to_null = str.maketrans('','',string.punctuation)
            q = q.translate(punct_to_null)
            a = a.translate(punct_to_null)
            q = q.strip().split(' ')
            a = a.strip().split(' ')
            q = q[1:]
            q = list(map(word2vec,q))
            questions.append(q)
            if len(q)>max_question_len:
                max_question_len = len(q)
            answers.append(list(map(vocab.index,a)))
            
        else: 
            punct_to_null = str.maketrans('','',string.punctuation)
            line = line.translate(punct_to_null)
            fact = line.strip().split(' ') 
            fact = fact[1:]
            fact = list(map(word2vec,fact))
            fact_story.append(fact)
            if len(fact)>max_fact_len:
                max_fact_len=len(fact)

        if flag_end_story == 1: 
            fact_stories.append(fact_story)  
            fact_story = [] 
            
    file.close()
        
    return fact_stories,questions,answers,max_fact_len,max_question_len

fact_stories,questions,answers,max_fact_len,max_question_len = extract_info(filename,max_fact_len,max_question_len)

filename = 'qa16_basic-induction_test.txt' 

test_fact_stories,test_questions,test_answers,max_fact_len,max_question_len = extract_info(filename,max_fact_len,max_question_len)


In [24]:
print(max_fact_len)
print(max_question_len)

4
4


In [25]:
print(map(vec2word,fact_stories[0][0]))

In [26]:
print(list(map(vec2word,test_fact_stories[0][0])))

['lily', 'is', 'a', 'swan']


In [27]:
PAD = word2vec('<PAD>')

for i in range(0,len(questions)):
    questions_len = len(questions[i])
    for j in range(questions_len,max_question_len):
        questions[i].append(PAD)
    for j in range(0,len(fact_stories[i])):
        fact_len = len(fact_stories[i][j])
        for k in range(fact_len,max_fact_len):
            fact_stories[i][j].append(PAD)
 

In [33]:
print(list(map(vec2word,fact_stories[0][2])))
print(list(map(vec2word,test_fact_stories[0][2])))

['bernhard', 'is', 'green', '<PAD>']
['greg', 'is', 'a', 'swan']
[array([-0.34441  ,  0.049968 ,  0.51707  , -1.231    , -0.093411 ,
       -0.18565  ,  0.0699   , -0.52303  , -0.82522  , -1.1475   ,
        0.34674  , -0.1359   , -0.017984 ,  0.66809  ,  0.12774  ,
       -0.37846  ,  0.025387 ,  0.22753  , -1.0253   ,  0.93013  ,
       -0.2632   ,  0.48023  ,  0.28662  ,  0.53986  ,  1.1494   ,
       -0.43907  ,  0.16174  , -0.91663  , -0.08995  ,  0.38935  ,
       -0.49759  ,  0.14868  ,  0.15115  ,  0.28914  ,  0.41117  ,
       -0.22251  , -0.54574  ,  0.42889  ,  0.28374  ,  0.61235  ,
       -0.15824  ,  0.24076  ,  0.55164  ,  0.45951  , -0.22147  ,
        0.2297   , -0.66727  , -0.89064  ,  0.087901 , -0.86329  ,
       -0.22344  ,  0.49091  , -0.66955  , -0.42588  ,  0.50533  ,
       -1.8709   , -0.24871  ,  0.1457   ,  0.1216   ,  0.40008  ,
        0.09388  , -0.2033   , -0.1014   ,  0.1109   ,  0.080344 ,
       -0.83206  ,  0.83037  ,  0.61361  ,  0.076019 ,  0.67569

In [29]:
for i in range(0,len(test_questions)):
    questions_len = len(test_questions[i])
    for j in range(questions_len,max_question_len):
        test_questions[i].append(PAD)
    for j in range(0,len(test_fact_stories[i])):
        fact_len = len(test_fact_stories[i][j])
        for k in range(fact_len,max_fact_len):
            test_fact_stories[i][j].append(PAD)

In [30]:
print(list(map(vec2word,test_fact_stories[0][3])))

['bernhard', 'is', 'white', '<PAD>']


In [35]:
fact_stories = np.asarray(fact_stories,np.float32)
print(fact_stories.shape)
questions = np.asarray(questions,np.float32)
print(questions.shape)
answers = np.asarray(answers,np.float32)
print(answers.shape)
test_fact_stories = np.asarray(test_fact_stories,np.float32)
print(test_fact_stories.shape)
test_questions = np.asarray(test_questions,np.float32)
print(test_questions.shape)
test_answers = np.asarray(test_answers,np.float32)
print(test_answers.shape)

(1000, 9, 4, 100)
(1000, 4, 100)


TypeError: float() argument must be a string or a number, not 'map'

In [ ]:
#Saving processed data in another file.

import pickle

PICK = [fact_stories,questions,answers,test_fact_stories,test_questions,test_answers]

with open('embeddingPICKLE', 'wb') as fp:
    pickle.dump(PICK, fp)